# Investment Decision
## Author: Snigdhayan Mahanta

`Monte Carlo Simulation` can be a powerful technique to get a quantitive idea about future outcomes. Suppose an investor has 10000 EUR to invest in a project (e.g., a startup). For the investor there are two options:
1. Option 1 - low rate of return and low risk of failure
2. Option 2 - high rate of return and high risk of failure

I used `Monte Carlo Simulation` using the `MonteCarlo` package of `R` to compute the average returns for the two options.

In [1]:
library(MonteCarlo, quietly = TRUE)

In [2]:
# Define a function that calculates the ROI of an investment based on rate of return and risk of failure
ROI <- function(rate, risk, investment){
  # generate investment outcome:
  outcome <- sample(0:1, size=1, prob=c((risk/100), 1-(risk/100))) # 0 = failure, 1 = success
  
  # calculate ROI:
  result <- ((rate/100)*investment*outcome)
  
  # return result:
  return(list("ROI" = result))
}

In [3]:
# define parameter grid:
investment <- 10000 # 10k EUR to invest

# Rate of return as percentage
low_rate <- 800 # 800% rate of return
high_rate <- 1200 # 1200% rate of return

# Risk of failure as percentage
low_risk <- 70 # 70% risk of failure
high_risk <- 80 # 80% risk of failure

rate <- c(low_rate, high_rate)
risk <- c(low_risk, high_risk)

In [4]:
# collect parameter grids in list:
param_list=list("rate" = rate, "risk" = risk, "investment" = investment)

In [5]:
# run simulation:
repetitions <- 100
MC_result<-MonteCarlo(func = ROI, nrep = repetitions, param_list = param_list)
summary(MC_result)

Grid of  4  parameter constellations to be evaluated. 
 
Progress: 
 
  |======================================================================| 100%
 
Simulation of function: 

function(rate, risk, investment){
  # generate investment outcome:
  outcome <- sample(0:1, size=1, prob=c((risk/100), 1-(risk/100))) # 0 = failure, 1 = success
  
  # calculate ROI:
  result <- ((rate/100)*investment*outcome)
  
  # return result:
  return(list("ROI" = result))
}
<bytecode: 0x000000001f80f4f8>

Required time: 0.06 secs for nrep = 100  repetitions on 1 CPUs 

Parameter grid: 

       rate : 800 1200 
       risk : 70 80 
 investment : 10000 

 
1 output arrays of dimensions: 2 2 1 100

In [6]:
# Aggregate results
return1 <- numeric(repetitions) # low rate, low risk
return2 <- numeric(repetitions) # high rate, high risk

for (i in c(1:repetitions)) {
    return1[i] <- MC_result$results$ROI[4*(i-1)+1]
    return2[i] <- MC_result$results$ROI[4*(i-1)+4]
}

In [7]:
# Average return - low rate and low risk
print(paste0("Average return ", "(Rate = ", low_rate, "%, Risk = ", low_risk, "%): ", mean(return1)))

[1] "Average return (Rate = 800%, Risk = 70%): 24800"


In [8]:
# Average return - high rate and high risk
print(paste0("Average return ", "(Rate = ", high_rate, "%, Risk = ", high_risk, "%): ", mean(return2)))

[1] "Average return (Rate = 1200%, Risk = 80%): 28800"


One can easily change the simulation parameters and analyze different scenarios. For an individual investor this information is merely of academic interest. However, this simulation information can be helpful for a venture capital firm that can invest 10000 EUR in many startups.